In [7]:
from time import time

import torch
import sklearn.datasets
import sklearn.preprocessing
import sklearn.model_selection
import numpy as np

import Onlinehd

np.random.seed(1234)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
# loads simple mnist dataset
def load():
    # fetches data
    x, y = sklearn.datasets.fetch_openml('mnist_784', return_X_y=True)
    x = x.astype(np.float)
    y = y.astype(np.int)
    y = np.array(y)

    # split and normalize
    x, x_test, y, y_test = sklearn.model_selection.train_test_split(x, y)
    scaler = sklearn.preprocessing.Normalizer().fit(x)
    x = scaler.transform(x)
    x_test = scaler.transform(x_test)

    # changes data to pytorch's tensors
    x = torch.from_numpy(x).float()
    y = torch.from_numpy(y).long()
    x_test = torch.from_numpy(x_test).float()
    y_test = torch.from_numpy(y_test).long()

    return x, x_test, y, y_test, scaler

In [3]:
# simple OnlineHD training

print('Loading...')
train_x, x_test, train_y, y_test, scaler = load()

Loading...


/var/folders/4_/4812b_4j2_s0smfmxlqdpr600000gn/T/ipykernel_99387/3995914916.py:5: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = x.astype(np.float)
/var/folders/4_/4812b_4j2_s0smfmxlqdpr600000gn/T/ipykernel_99387/3995914916.py:6: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://nu

In [8]:
from time import time

classes = train_y.unique().size(0)
features = train_x.size(1)
model = Onlinehd.OnlineHD(classes, features, dim=10000)

if torch.cuda.is_available():
    train_x = train_x.cuda()
    train_y = train_y.cuda()
    x_test = x_test.cuda()
    y_test = y_test.cuda()
    model = model.to('cuda')
    print('Using GPU!')

print('Training...')
t = time()
model = model.fit(train_x, train_y, bootstrap=1.0, lr=0.035, epochs=20)
t = time() - t

print('Validating...')
yhat = model(train_x)
yhat_test = model(x_test)
acc = (train_y == yhat).float().mean()
acc_test = (y_test == yhat_test).float().mean()
print(f'{acc = :6f}')
print(f'{acc_test = :6f}')
print(f'{t = :6f}')

Training...
0
10
Validating...
acc = 0.944476
acc_test = 0.933829
t = 27.034414


In [9]:
def pgd_attack(x_original, y_target, model, alpha, iterations=10, epsilon=0.03):
    criterion = nn.CosineEmbeddingLoss()
    x_perturbed = x_original.clone().detach()
    x_perturbed.requires_grad = True
    
    for _ in range(iterations):
        output = model.encode(x_perturbed)
        label = torch.ones(10, dtype=int) * -1
        label[y_target.item()] = 1
        loss = criterion(output, model.model, label)
                
#         loss.backward()
#         grad = x_perturbed.gard.data
        grad = torch.autograd.grad(loss, x_perturbed)[0]
        x_perturbed = x_perturbed.detach() + alpha*grad.sign()
        eta = torch.clamp(x_perturbed - x_original, min=-epsilon, max=epsilon)
        x_perturbed = x_original + eta
    return x_perturbed

In [15]:
import time
import torch.nn as nn

epsilons = torch.Tensor([0.01, 0.02, 0.03, 0.04, 0.05])
acc_results = dict()
verbose = False
N_VAL_SAMPLES = x_test.shape[0]
criterion = nn.CosineEmbeddingLoss()

PGD_test_attack_data = []
PGD_test_attack_label = []

for eps in epsilons:
    correct_unperturbed = 0
    correct_perturbed = 0
    t0 = time.perf_counter()
    
    for j in range(len(x_test)):
        if j % 1000 == 0 :
            print(j)
#     for j, val_data in enumerate(x, 0):
        ### NOTE: IT WOULD BE MORE EFFICIENT TO ITERATE ONLY ONCE THROUGH THE DATA AND PERFORM ALL THE ATTACKS
        x, y_target = x_test[j].reshape(-1, 784), y_test[j]
        x, y_target = x.to(device), y_target.to(device)
        x.requires_grad = True
        
        y_pred = model(x)
        
        if y_pred == y_target: # Only make attack on correctly classified samples
            correct_unperturbed += 1
            # Calculate loss and gradient
            perturbed_x = pgd_attack(x, y_target, model=model, alpha=eps)
            PGD_test_attack_data.append(perturbed_x)
            PGD_test_attack_label.append(y_target)
            
            y_pred_perturbed = model(perturbed_x)
#             y_pred_perturbed = torch.argmin(perturbed_output)
#             loss_perturbed = criterion(perturbed_output, model.model, label)
            if y_pred_perturbed == y_target:
                correct_perturbed += 1
                
    acc_before_attack = correct_unperturbed / N_VAL_SAMPLES
    acc_after_attack = correct_perturbed / N_VAL_SAMPLES
    print(f'\nPGD Attack with alpha = {eps:.5f} | Elapsed time: {time.perf_counter() - t0:.2f} seconds.')
    print(f'Accuracy: Before the attack -> {100 * acc_before_attack:.2f}%\t|\tAfter the attack -> {100 * acc_after_attack:.2f}%')
    acc_results[eps.item()] = acc_after_attack
acc_results[0] = acc_before_attack

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000

PGD Attack with alpha = 0.01000 | Elapsed time: 808.72 seconds.
Accuracy: Before the attack -> 93.38%	|	After the attack -> 46.13%
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000

PGD Attack with alpha = 0.02000 | Elapsed time: 791.69 seconds.
Accuracy: Before the attack -> 93.38%	|	After the attack -> 50.78%
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000

PGD Attack with alpha = 0.03000 | Elapsed time: 794.20 seconds.
Accuracy: Before the attack -> 93.38%	|	After the attack -> 62.66%
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000

PGD Attack with alpha = 0.04000 | Elapsed time: 795.21 seconds.
Accuracy: Before the attack -> 93.38%	|	After the attack -> 65.10%
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
170

In [19]:
PGD_test_attack_label
for i in range(len(PGD_test_attack_label)):
    PGD_test_attack_label[i] = PGD_test_attack_label[i].item()

In [23]:
PGD_test_attack_label = torch.tensor(PGD_test_attack_label).long()

/var/folders/4_/4812b_4j2_s0smfmxlqdpr600000gn/T/ipykernel_99387/3997804931.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  PGD_test_attack_label = torch.tensor(PGD_test_attack_label).long()


In [25]:
len(PGD_test_attack_data)

81710

In [26]:
PGD_test_attack = torch.zeros(81710, 784)

for i in range(len(PGD_test_attack_data)):
    PGD_test_attack[i] = PGD_test_attack_data[i].reshape(784)

In [30]:
train_data = scaler.transform(PGD_test_attack.detach().numpy())

In [31]:
train_data = torch.from_numpy(train_data).float()

# Adversarial Training

In [32]:
from time import time

classes = train_y.unique().size(0)
features = train_x.size(1)
model = onlinehd.OnlineHD(classes, features, dim=10000)

if torch.cuda.is_available():
    train_x = train_x.cuda()
    train_y = train_y.cuda()
    x_test = x_test.cuda()
    y_test = y_test.cuda()
    model = model.to('cuda')
    print('Using GPU!')

print('Training...')
t = time()
model = model.fit(train_data, PGD_test_attack_label, bootstrap=1.0, lr=0.035, epochs=20)
t = time() - t

In [34]:
print('Validating...')
yhat = model(train_data)
yhat_test = model(x_test)
acc = (PGD_test_attack_label == yhat).float().mean()
acc_test = (y_test == yhat_test).float().mean()
print(f'{acc = :6f}')
print(f'{acc_test = :6f}')
print(f'{t = :6f}')

Validating...
acc = 0.992877
acc_test = 0.882343
t = 28.965304


In [36]:
import pickle

with open('hd_adversarial_sample/MNIST_HD_FGSM.pickle', 'rb') as f:
    FGSM = pickle.load(f)

FGSM_001 = FGSM['data']['0.01']
FGSM_003 = FGSM['data']['0.03']
FGSM_007 = FGSM['data']['0.07']
FGSM_01 = FGSM['data']['0.1']

import pickle

with open('hd_adversarial_sample/MNIST_HD_DF.pickle', 'rb') as f:
    DF = pickle.load(f)

DF_data = DF['data']

import pickle

with open('hd_adversarial_sample/MNIST_HD_JSMA.pickle', 'rb') as f:
    JSMA = pickle.load(f)

JSMA_data = JSMA['data']

In [48]:
FGSM_001.shape

torch.Size([17500, 28, 28])

In [50]:
FGSM_001 = torch.from_numpy(scaler.transform(FGSM_001.detach().numpy().reshape(-1, 784))).float()
FGSM_003 = torch.from_numpy(scaler.transform(FGSM_003.detach().numpy().reshape(-1, 784))).float()
FGSM_007 = torch.from_numpy(scaler.transform(FGSM_007.detach().numpy().reshape(-1, 784))).float()
FGSM_01 = torch.from_numpy(scaler.transform(FGSM_01.detach().numpy().reshape(-1, 784))).float()
DF_data = torch.from_numpy(scaler.transform(DF_data.detach().numpy().reshape(-1, 784))).float()
JSMA_data = torch.from_numpy(scaler.transform(JSMA_data.detach().numpy().reshape(-1, 784))).float()

In [52]:
print('Validating...')

FGSM_001_yhat = model(FGSM_001)
FGSM_003_yhat = model(FGSM_003)
FGSM_007_yhat = model(FGSM_007)
FGSM_01_yhat = model(FGSM_01)
DF_data_yhat = model(DF_data)
JSMA_data_yhat = model(JSMA_data)

FGSM_001_acc = (y_test == FGSM_001_yhat).float().mean()
FGSM_003_acc = (y_test == FGSM_003_yhat).float().mean()
FGSM_007_acc = (y_test == FGSM_007_yhat).float().mean()
FGSM_01_acc = (y_test == FGSM_01_yhat).float().mean()
DF_acc = (y_test == DF_data_yhat).float().mean()
JSMA_acc = (y_test == JSMA_data_yhat).float().mean()

print(f'{FGSM_001_acc = :6f}')
print(f'{FGSM_003_acc = :6f}')
print(f'{FGSM_007_acc = :6f}')
print(f'{FGSM_01_acc = :6f}')
print(f'{DF_acc = :6f}')
print(f'{JSMA_acc = :6f}')

Validating...
FGSM_001_acc = 0.889771
FGSM_003_acc = 0.879886
FGSM_007_acc = 0.759543
FGSM_01_acc = 0.634343
DF_acc = 0.876971
JSMA_acc = 0.846229
